# Exploring and cleaning the data from "Chronik flüchtlingsfeindlicher Vorfälle"

In [1]:
import numpy as np
import pandas as pd

## Load data and get first overview

Check the data set.

In [2]:
df = pd.read_csv('data/mut_gegen_rechte_gewalt.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9012 entries, 0 to 9011
Data columns (total 8 columns):
date           9012 non-null object
category       9012 non-null object
city           9012 non-null object
bundesland     9012 non-null object
casualties     611 non-null object
description    9012 non-null object
source         8258 non-null object
source_url     8259 non-null object
dtypes: object(8)
memory usage: 563.3+ KB


In [4]:
df.head()

,date,category,city,bundesland,casualties,description,source,source_url
0,17.05.2019,Tätlicher Übergriff/Körperverletzung,Prenzlau,Brandenburg,1Verletzte_r,Zwei Deutsche haben am Abend zunächst neben ei...,Nordkurier,{'https://www.nordkurier.de/uckermark/junge-ma...
1,04.05.2019,Tätlicher Übergriff/Körperverletzung,Querfurt,Sachsen-Anhalt,NaN,Ein 21-jähriger aus Syrien wurde in der Nacht ...,Mitteldeutsche Zeitung,{'https://www.mz-web.de/saalekreis/staatsschut...
2,01.05.2019,Sonstige Angriffe,Kirchheim,Hessen,NaN,Unbekannte haben in der Nacht Eier gegen die F...,Süddeutsche Zeitung,{'https://www.sueddeutsche.de/muenchen/staatss...
3,31.03.2019,Verdachtsfall,Lübeck,Schleswig-Holstein,1Verletzte_r,Zwei unbekannte Männer haben einen 27-jährigen...,n-tv,{'https://www.n-tv.de/regionales/hamburg-und-s...
4,02.03.2019,Tätlicher Übergriff/Körperverletzung,Leipzig,Sachsen,1Verletzte_r,Eine Gruppe von acht Männern hat am Nachmittag...,Peiner Allgemeine,{'http://www.paz-online.de/Nachrichten/Panoram...


## Data cleaning

In [5]:
df.dtypes

date           object
category       object
city           object
bundesland     object
casualties     object
description    object
source         object
source_url     object
dtype: object

### Date

Change date column to datetime object.

In [6]:
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')

In [7]:
df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

### Category

In [8]:
df['category'].value_counts()

Sonstige Angriffe                       6611
Tätlicher Übergriff/Körperverletzung    1482
Kundgebung/Demo                          361
Verdachtsfall                            286
Brandanschlag                            272
Name: category, dtype: int64

As the data has been scraped from a German website, the five category names are in German. We will replace the German categories with the English translation.

In [9]:
mapping_dict = {
    'Tätlicher Übergriff/Körperverletzung': 'Assault and battery',
    'Brandanschlag': 'Arson attack',
    'Kundgebung/Demo': 'Rally/demonstration',
    'Sonstige Angriffe': 'Other attacks',
    'Verdachtsfall': 'Suspected case'
}
df['category'] = df['category'].map(mapping_dict)

In [10]:
df['category'].value_counts()

Other attacks          6611
Assault and battery    1482
Rally/demonstration     361
Suspected case          286
Arson attack            272
Name: category, dtype: int64

### City

In [11]:
df['city'].unique().shape[0]

2505

### Bundesland

In [12]:
df['bundesland'].unique().shape[0]

16

### Casualties

In [13]:
df['casualties'].head()

0    1Verletzte_r
1             NaN
2             NaN
3    1Verletzte_r
4    1Verletzte_r
Name: casualties, dtype: object

How many entries actually do contain casualty numbers?

In [14]:
df[df['casualties'].notnull()].shape[0]

611

There are only 611 entries which contain information on the number of casualties. I suppose this can be interpreted than in other cases there were (luckily) no casualties. 

Let's replace the string representation into numerical values by deleting the characters.

In [15]:
df['casualties'] = df['casualties'].str.replace(r' ?(Verletzte)(_r)?', '').astype(float)

In [16]:
df['casualties'].head()

0    1.0
1    NaN
2    NaN
3    1.0
4    1.0
Name: casualties, dtype: float64

### Description

In [17]:
print(*df['description'].head(3), sep='\n\n')

Zwei Deutsche haben am Abend zunächst neben einer Asylunterkunft randaliert. Als Kinder, die in der Asylunterkunft leben, sie aufforderten, dies zu unterlassen, betraten die beiden 21- bzw. 23-Jährigen das Gelände der Unterkunft. Einer von ihnen zückte ein Messer und soll laut Polizei "Stichbewegungen gegen einen tschetschenischen Bewohner ausgeführt haben. Bei der folgenden Rangelei verletzte sich der Tschetschene an der Hand, ein Deutscher erlitt Verletzungen am Bein und musste operiert werden", so die Polizei weiter. Die Kriminalpolizei ermittelt.

Ein 21-jähriger aus Syrien wurde in der Nacht aus einer Gruppe aus fünf oder sechs jungen Deutschen zunächst rassistisch beleidigt und dann auch geschlagen. Als ein 47-jähriger Zeuge dazwischengehen wollte, sollen ihn die Angreifer zurückgestoßen und am Fuß verletzt haben. Der 21-Jährige musste nicht behandelt werden. Die Täter flüchteten, der Staatsschutz ermittelt.

Unbekannte haben in der Nacht Eier gegen die Fassade einer Asylunterkun

### Source

In [18]:
df['source'].head()

0                Nordkurier
1    Mitteldeutsche Zeitung
2       Süddeutsche Zeitung
3                      n-tv
4         Peiner Allgemeine
Name: source, dtype: object

In [19]:
df['source'].unique().shape[0]

596

### Source url

In [20]:
print(*df['source_url'].head(), sep='\n')

{'https://www.nordkurier.de/uckermark/junge-maenner-in-prenzlau-randalieren-1935542005.html'}
{'https://www.mz-web.de/saalekreis/staatsschutz-ermittelt-junger-syrer-rassistisch-beschimpft-und-attackiert-32472432'}
{'https://www.sueddeutsche.de/muenchen/staatsschutz-ermittelt-angriffe-auf-fluechtlinge-und-ein-drohbrief-1.4429977'}
{'https://www.n-tv.de/regionales/hamburg-und-schleswig-holstein/Syrer-mit-Glasflasche-attackiert-Fremdenfeindliches-Motiv-article20957339.html'}
{'http://www.paz-online.de/Nachrichten/Panorama/Auslaenderfeindlicher-Attacke-Acht-betrunkene-Maenner-verpruegeln-Asylbewerber'}


Delete the curly brackets:

In [21]:
df['source_url'] = df['source_url'].str.replace(r"(\{')?('\})?", '')

In [22]:
print(*df['source_url'].head(), sep='\n')

https://www.nordkurier.de/uckermark/junge-maenner-in-prenzlau-randalieren-1935542005.html
https://www.mz-web.de/saalekreis/staatsschutz-ermittelt-junger-syrer-rassistisch-beschimpft-und-attackiert-32472432
https://www.sueddeutsche.de/muenchen/staatsschutz-ermittelt-angriffe-auf-fluechtlinge-und-ein-drohbrief-1.4429977
https://www.n-tv.de/regionales/hamburg-und-schleswig-holstein/Syrer-mit-Glasflasche-attackiert-Fremdenfeindliches-Motiv-article20957339.html
http://www.paz-online.de/Nachrichten/Panorama/Auslaenderfeindlicher-Attacke-Acht-betrunkene-Maenner-verpruegeln-Asylbewerber


### Final checkup

In [23]:
df.dtypes

date           datetime64[ns]
category               object
city                   object
bundesland             object
casualties            float64
description            object
source                 object
source_url             object
day                     int64
month                   int64
year                    int64
dtype: object

In [24]:
df.head()

,date,category,city,bundesland,casualties,description,source,source_url,day,month,year
0,2019-05-17,Assault and battery,Prenzlau,Brandenburg,1.0,Zwei Deutsche haben am Abend zunächst neben ei...,Nordkurier,https://www.nordkurier.de/uckermark/junge-maen...,17,5,2019
1,2019-05-04,Assault and battery,Querfurt,Sachsen-Anhalt,NaN,Ein 21-jähriger aus Syrien wurde in der Nacht ...,Mitteldeutsche Zeitung,https://www.mz-web.de/saalekreis/staatsschutz-...,4,5,2019
2,2019-05-01,Other attacks,Kirchheim,Hessen,NaN,Unbekannte haben in der Nacht Eier gegen die F...,Süddeutsche Zeitung,https://www.sueddeutsche.de/muenchen/staatssch...,1,5,2019
3,2019-03-31,Suspected case,Lübeck,Schleswig-Holstein,1.0,Zwei unbekannte Männer haben einen 27-jährigen...,n-tv,https://www.n-tv.de/regionales/hamburg-und-sch...,31,3,2019
4,2019-03-02,Assault and battery,Leipzig,Sachsen,1.0,Eine Gruppe von acht Männern hat am Nachmittag...,Peiner Allgemeine,http://www.paz-online.de/Nachrichten/Panorama/...,2,3,2019
